In [2]:
import json, time, requests, pandas as pd, re

In [273]:
# 과학적 표기법 사용안함
pd.options.display.float_format = '{:.2f}'.format

In [277]:
# API로 데이터 불러오기
start_time = time.time()
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
api_url = 'https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit=2&offset=1&region=north_america'
api = requests.get(api_url, headers=headers)
print(f"{time.time()-start_time} s")

0.5472970008850098 s


In [278]:
start_time = time.time()
data = json.loads(api.text)
print(f"{time.time()-start_time} s")

0.00013637542724609375 s


In [279]:
data

{'data': {'filters': None,
  'table': {'headers': {'symbol': 'Symbol',
    'name': 'Name',
    'lastsale': 'Last Sale',
    'netchange': 'Net Change',
    'pctchange': '% Change',
    'marketCap': 'Market Cap'},
   'rows': [{'symbol': 'MSFT',
     'name': 'Microsoft Corporation Common Stock',
     'lastsale': '$247.95',
     'netchange': '-1.73',
     'pctchange': '-0.693%',
     'marketCap': '1,867,453,909,920',
     'url': '/market-activity/stocks/msft'},
    {'symbol': 'AMZN',
     'name': 'Amazon.com, Inc. Common Stock',
     'lastsale': '$3222.09',
     'netchange': '-0.98',
     'pctchange': '-0.03%',
     'marketCap': '1,624,976,466,528',
     'url': '/market-activity/stocks/amzn'}]},
  'totalrecords': 6149,
  'asof': 'Last price as of Jun 01, 2021'},
 'message': None,
 'status': {'rCode': 200, 'bCodeMessage': None, 'developerMessage': None}}

In [280]:
# 상장기업수 체크
start_time = time.time()
num_of_listed_comp = data['data']['totalrecords']
print(f"{time.time()-start_time} s")

7.796287536621094e-05 s


In [281]:
start_time = time.time()
num_of_listed_comp = data.get('data').get('totalrecords') # which is faster?
print(f"{time.time()-start_time} s")

9.965896606445312e-05 s


In [282]:
print(num_of_listed_comp)

6149


In [293]:
# 주가 데이터
d = data['data']['table']['rows']

In [312]:
start_time = time.time()
df = pd.DataFrame(d)
print(f"{time.time()-start_time} s")

0.0010752677917480469 s


In [313]:
df

,symbol,name,lastsale,netchange,pctchange,marketCap,url
0,MSFT,Microsoft Corporation Common Stock,$247.95,-1.73,-0.693%,"1,867,453,909,920",/market-activity/stocks/msft
1,AMZN,"Amazon.com, Inc. Common Stock",$3222.09,-0.98,-0.03%,"1,624,976,466,528",/market-activity/stocks/amzn


In [314]:
df.iloc[:,2:6]

,lastsale,netchange,pctchange,marketCap
0,$247.95,-1.73,-0.693%,"1,867,453,909,920"
1,$3222.09,-0.98,-0.03%,"1,624,976,466,528"


In [315]:
def convert_to_float(data):
    # 2~6열 까지 데이터
    for i in range(2, 6):
        # 정규표현식 : . 을 제외한 나머지 숫자외의 것들을 제거
        data.iloc[:,i] = data.iloc[:,i].apply(lambda x: re.sub('[^\d\.]', '', x)).astype(float)

In [316]:
# 문자열로 되어있는 numeric 데이터를 실수로 변환
convert_to_float(df)

In [317]:
df

,symbol,name,lastsale,netchange,pctchange,marketCap,url
0,MSFT,Microsoft Corporation Common Stock,247.95,1.73,0.69,1867453909920.00,/market-activity/stocks/msft
1,AMZN,"Amazon.com, Inc. Common Stock",3222.09,0.98,0.03,1624976466528.00,/market-activity/stocks/amzn
